# Basic CLIP Example
A lot of this code is copied from my repo: https://github.com/berkott/cvInterp

Also, upload bunny.zip from the paper

CLIP repo + download instructions: https://github.com/openai/CLIP

In [3]:
import torch
import clip
from PIL import Image
import torchvision.transforms as T
from absl import logging
import numpy as np
from six.moves import range
# from sklearn import svm
# import gin.tf
import os
import random
import scipy.stats
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.svm import LinearSVC
from sklearn.metrics import mutual_info_score, roc_auc_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
#TODO:

#Get Bunny images
#Get pipieline
#Test SAP
#TEst NIG

#Get Geomancer
#G


#Do different VAE sizes (find different B-VAE)

ImportError: /home/berk/anaconda3/envs/python38/lib/python3.8/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/berk/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/optimize/_highs/_highs_wrapper.cpython-38-x86_64-linux-gnu.so)

In [ ]:
clip.clip._MODELS = {
    "RN50": "https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt",
    "RN101": "https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt",
    "RN50x4": "https://openaipublic.azureedge.net/clip/models/7e526bd135e493cef0776de27d5f42653e6b4c8bf9e0f653bb11773263205fdd/RN50x4.pt",
    "RN50x16": "https://openaipublic.azureedge.net/clip/models/52378b407f34354e150460fe41077663dd5b39c54cd0bfd2b27167a4a06ec9aa/RN50x16.pt",
    "RN50x64": "https://openaipublic.azureedge.net/clip/models/be1cfb55d75a9666199fb2206c106743da0f6468c9d327f3e0d0a543a9919d9c/RN50x64.pt",
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",
    "ViT-B/16": "https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt",
    "ViT-L/14": "https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt",
    "ViT-L/14@336px": "https://openaipublic.azureedge.net/clip/models/3035c92b350959924f9f00213499208652fc7ea050643e8b385c2dac08641f02/ViT-L-14-336px.pt",
}

MODELS_TO_USE = [ "RN50", "RN101", "RN50x4"]


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device, jit=False)

## Model Param Details
I think we should just start by running the experiments on RN50, RN101, and RN50x4. These are 3 resnet architectures that are basically just scaled up to different sizes. Then if we have extra time, we cna test on everything. 

In [ ]:
# Get model param details
# for name in clip.clip._MODELS_TO_USE:
#     m, _ = clip.load(name, device=device, jit=False)
#     print(name)
#     print(f"Num Params: {sum(p.numel() for p in m.parameters())}")
#     print(m)

In [ ]:
image_paths = os.listdir('./drive/MyDrive/images')
images = []
for path in image_paths[:1000]:
  images.append(preprocess(Image.open("./drive/MyDrive/images/" + path)).unsqueeze(0).to(device)) # TODO: Specify correct image here!!

# T.ToPILImage()(image[0]).show()


# text = clip.tokenize(["bunny"]).to(device)
# image_encodings = []
# for image in images:
#   with torch.no_grad():
#       image_encodings.append(model.encode_image(image))
#       text_features = model.encode_text(text)

#       # THIS IMAGE FEATURES VECTOR IS THE LATENT REPRESENTATION WE WANT!!!
#       # print(image_features.shape)
#       # print(image_features)
      
#       logits_per_image, logits_per_text = model(image, text)
#       probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#   print("Label probs:", probs)

# Mine

In [ ]:
def load_dSprites(path):
    # part of the code is from:
    # https://github.com/deepmind/dsprites-dataset/blob/master/dsprites_reloading_example.ipynb
    dataset_zip = np.load(
        os.path.join(
            path, "dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz"),
        allow_pickle=True)
    imgs = dataset_zip['imgs']
    latent_values = dataset_zip['latents_values']
    #latents_classes = dataset_zip['latents_classes']
    metadata = dataset_zip['metadata'][()]

    imgs = imgs.reshape(737280, 64, 64, 1).astype(np.float)  # 0 ~ 1

    latents_names = metadata["latents_names"]
    latents_sizes = metadata["latents_sizes"]
    latents_possible_values = metadata["latents_possible_values"]
    latents_bases = np.concatenate(
        (latents_sizes[::-1].cumprod()[::-1][1:], np.array([1, ])))

    def latent_to_index(latents):
        return np.dot(latents, latents_bases).astype(int)

    def sample_latent(size=1):
        samples = np.zeros((size, latents_sizes.size))
        for lat_i, lat_size in enumerate(latents_sizes):
            samples[:, lat_i] = np.random.randint(lat_size, size=size)
        return samples

    metric_data_groups = []
    L = 100
    M = 500

    for i in range(M):
        fixed_latent_id = i % 5 + 1
        latents_sampled = sample_latent(size=L)
        latents_sampled[:, fixed_latent_id] = \
            np.random.randint(latents_sizes[fixed_latent_id], size=1)
        # print(latents_sampled[0:10])
        indices_sampled = latent_to_index(latents_sampled)
        imgs_sampled = imgs[indices_sampled]
        metric_data_groups.append(
            {"img": imgs_sampled,
             "label": fixed_latent_id - 1})

    selected_ids = np.random.permutation(range(imgs.shape[0]))
    selected_ids = selected_ids[0: imgs.shape[0] / 10]
    metric_data_eval_std = imgs[selected_ids]

    random_latent_ids = sample_latent(size=imgs.shape[0] / 10)
    random_latent_ids = random_latent_ids.astype(np.int32)
    random_ids = latent_to_index(random_latent_ids)
    assert random_latent_ids.shape == (imgs.shape[0] / 10, 6)
    random_imgs = imgs[random_ids]

    random_latents = np.zeros((random_imgs.shape[0], 6))
    for i in range(6):
        random_latents[:, i] = \
            latents_possible_values[latents_names[i]][random_latent_ids[:, i]]

    assert np.all(random_latents[:, 0] == 1)
    assert np.min(random_latents[:, 1]) == 1
    assert np.max(random_latents[:, 1]) == 3

    random_latents = random_latents[:, 1:]
    random_latents[:, 0] -= 1.0

    metric_data_img_with_latent = {
        "img": random_imgs,
        "latent": random_latents,
        "latent_id": random_latent_ids[:, 1:],
        "is_continuous": [False, True, True, True, True]}

    metric_data = {
        "groups": metric_data_groups,
        "img_eval_std": metric_data_eval_std,
        "img_with_latent": metric_data_img_with_latent}

    return imgs, metric_data, latent_values, metadata

In [ ]:
class Metric(object):
    # def __init__(self, sess):
    #     self.sess = sess
    #     self.model = None
    def __init__(self):
        self.model = None

    def set_model(self, model):
        self.model = model

    def build(self):
        pass

    def load(self):
        pass

    def evaluate(self, epoch_id, batch_id, global_id):
        raise NotImplementedError

class SAPMetric(Metric):
    """ Impementation of the metric in: 
        VARIATIONAL INFERENCE OF DISENTANGLED LATENT CONCEPTS FROM UNLABELED 
        OBSERVATIONS
        Part of the code is adapted from:
        https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/evaluation/metrics/sap_score.py
    """
    def __init__(self, metric_data, *args, **kwargs):
        super(SAPMetric, self).__init__(*args, **kwargs)
        self.metric_data = metric_data

    def evaluate(self, epoch_id, batch_id, global_id):
        data_inference = self.model.inference_from(
            self.metric_data["img_with_latent"]["img"])
        data_gt_latents = self.metric_data["img_with_latent"]["latent"]
        factor_is_continuous = \
            self.metric_data["img_with_latent"]["is_continuous"]

        num_latents = data_inference.shape[1]
        num_factors = len(factor_is_continuous)

        score_matrix = np.zeros([num_latents, num_factors])
        for i in range(num_latents):
            for j in range(num_factors):
                inference_values = data_inference[:, i]
                gt_values = data_gt_latents[:, j]
                if factor_is_continuous[j]:
                    cov = np.cov(inference_values, gt_values, ddof=1)
                    assert np.all(np.asarray(list(cov.shape)) == 2)
                    cov_cov = cov[0, 1]**2
                    cov_sigmas_1 = cov[0, 0]
                    cov_sigmas_2 = cov[1, 1]
                    score_matrix[i, j] = cov_cov / cov_sigmas_1 / cov_sigmas_2
                else:
                    gt_values = gt_values.astype(np.int32)
                    classifier = LinearSVC(C=0.01, class_weight="balanced")
                    classifier.fit(inference_values[:, np.newaxis], gt_values)
                    pred = classifier.predict(inference_values[:, np.newaxis])
                    score_matrix[i, j] = np.mean(pred == gt_values)
        sorted_score_matrix = np.sort(score_matrix, axis=0)
        score = np.mean(sorted_score_matrix[-1, :] - 
                        sorted_score_matrix[-2, :])

        return {"SAP_metric": score,
                "SAP_metric_detail": score_matrix}


class MIGMetric(Metric):
    """ Impementation of the metric in: 
        Isolating Sources of Disentanglement in Variational Autoencoders
        Part of the code is adapted from:
        https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/evaluation/metrics/mig.py
    """
    def __init__(self, metric_data, *args, **kwargs):
        super(MIGMetric, self).__init__(*args, **kwargs)
        self.metric_data = metric_data

    def discretize(self, data, num_bins=20):
        """ Adapted from:
            https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/evaluation/metrics/utils.py
        """
        discretized = np.zeros_like(data)
        for i in range(data.shape[1]):
            discretized[:, i] = np.digitize(
                data[:, i],
                np.histogram(data[:, i], num_bins)[1][:-1])
        return discretized

    def mutual_info(self, data1, data2):
        """ Adapted from:
            https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/evaluation/metrics/utils.py
        """
        n1 = data1.shape[1]
        n2 = data2.shape[1]
        mi = np.zeros([n1, n2])
        for i in range(n1):
            for j in range(n2):
                mi[i, j] = mutual_info_score(
                    data2[:, j], data1[:, i])
        return mi

    def entropy(self, data):
        """ Adapted from:
            https://github.com/google-research/disentanglement_lib/blob/master/disentanglement_lib/evaluation/metrics/utils.py
        """
        num_factors = data.shape[1]
        entr = np.zeros(num_factors)
        for i in range(num_factors):
            entr[i] = mutual_info_score(data[:, i], data[:, i])
        return entr

    def evaluate(self, epoch_id, batch_id, global_id):
        data_inference = self.model.inference_from(
            self.metric_data["img_with_latent"]["img"])
        data_gt_latents = self.metric_data["img_with_latent"]["latent_id"]

        data_inference_discrete = self.discretize(data_inference)
        mi = self.mutual_info(
            data_inference_discrete, data_gt_latents)
        entropy = self.entropy(data_gt_latents)
        sorted_mi = np.sort(mi, axis=0)[::-1]
        mig_score = np.mean(
            np.divide(sorted_mi[0, :] - sorted_mi[1, :], entropy))

        return {"MIG_metric": mig_score,
                "MIG_metric_detail_mi": mi,
                "MIG_metric_detail_entropy": entropy}

In [ ]:
results = {}

data, metric_data, latent_values, metadata = load_dSprites("")

sapMetric_f = SAPMetric(metric_data)
sapMetric_f.set_model(model.encode_image)
results["SAP"] = sapMetric_f.evaluate(-1, -1, -1)

migMetric_f = MIGMetric(metric_data)
migMetric_f.set_model(model.encode_image)
results["MIG"] = migMetric_f.evaluate(-1, -1, -1)

In [ ]:
from gpu_task_scheduler.gpu_task import GPUTask


class GANTask(GPUTask):
    def main(self):
        import os
        import tensorflow as tf
        from gan.load_data import load_dSprites
        from gan.latent import UniformLatent, JointLatent
        from gan.network import Decoder, InfoGANDiscriminator, \
            CrDiscriminator, MetricRegresser
        from gan.infogan_cr import INFOGAN_CR
        from gan.metric import FactorVAEMetric, DSpritesInceptionScore, \
            DHSICMetric, \
            BetaVAEMetric, SAPMetric, FStatMetric, MIGMetric, DCIMetric
        import pickle

        data, metric_data, latent_values, metadata = load_dSprites("")
        _, height, width, depth = data.shape

        latent_list = []

        for i in range(self._config["uniform_reg_dim"]):
            latent_list.append(UniformLatent(
                in_dim=1, out_dim=1, low=-1.0, high=1.0, q_std=1.0,
                apply_reg=True))
        if self._config["uniform_not_reg_dim"] > 0:
            latent_list.append(UniformLatent(
                in_dim=self._config["uniform_not_reg_dim"],
                out_dim=self._config["uniform_not_reg_dim"],
                low=-1.0, high=1.0, q_std=1.0,
                apply_reg=False))
        latent = JointLatent(latent_list=latent_list)

        # decoder = Decoder(
        #     output_width=width, output_height=height, output_depth=depth)
        # infoGANDiscriminator = \
        #     InfoGANDiscriminator(
        #         output_length=latent.reg_out_dim,
        #         q_l_dim=self._config["q_l_dim"])
        # crDiscriminator = CrDiscriminator(output_length=latent.num_reg_latent)

        # shape_network = MetricRegresser(
        #     output_length=3,
        #     scope_name="dSpritesSampleQualityMetric_shape")

        # checkpoint_dir = os.path.join(self._work_dir, "checkpoint")
        # if not os.path.exists(checkpoint_dir):
        #     os.makedirs(checkpoint_dir)
        # sample_dir = os.path.join(self._work_dir, "sample")
        # if not os.path.exists(sample_dir):
        #     os.makedirs(sample_dir)
        # time_path = os.path.join(self._work_dir, "time.txt")
        # metric_path = os.path.join(self._work_dir, "metric.csv")

        # run_config = tf.ConfigProto()
        # run_config.gpu_options.allow_growth = True
        # with tf.Session(config=run_config) as sess:
        #     factorVAEMetric = FactorVAEMetric(metric_data, sess=sess)
        #     dSpritesInceptionScore = DSpritesInceptionScore(
        #         sess=sess,
        #         do_training=False,
        #         data=data,
        #         metadata=metadata,
        #         latent_values=latent_values,
        #         network_path="metric_model/DSprites",
        #         shape_network=shape_network,
        #         sample_dir=sample_dir)
        #     dHSICMetric = DHSICMetric(
        #         sess=sess,
        #         data=data)
        #     metric_callbacks = [factorVAEMetric,
        #                         dSpritesInceptionScore,
        #                         dHSICMetric]
        #     gan = INFOGAN_CR(
        #         sess=sess,
        #         checkpoint_dir=checkpoint_dir,
        #         sample_dir=sample_dir,
        #         time_path=time_path,
        #         epoch=self._config["epoch"],
        #         batch_size=self._config["batch_size"],
        #         data=data,
        #         vis_freq=self._config["vis_freq"],
        #         vis_num_sample=self._config["vis_num_sample"],
        #         vis_num_rep=self._config["vis_num_rep"],
        #         latent=latent,
        #         decoder=decoder,
        #         infoGANDiscriminator=infoGANDiscriminator,
        #         crDiscriminator=crDiscriminator,
        #         gap_start=self._config["gap_start"],
        #         gap_decrease_times=self._config["gap_decrease_times"],
        #         gap_decrease=self._config["gap_decrease"],
        #         gap_decrease_batch=self._config["gap_decrease_batch"],
        #         cr_coe_start=self._config["cr_coe_start"],
        #         cr_coe_increase_times=self._config["cr_coe_increase_times"],
        #         cr_coe_increase=self._config["cr_coe_increase"],
        #         cr_coe_increase_batch=self._config["cr_coe_increase_batch"],
        #         info_coe_de=self._config["info_coe_de"],
        #         info_coe_infod=self._config["info_coe_infod"],
        #         metric_callbacks=metric_callbacks,
        #         metric_freq=self._config["metric_freq"],
        #         metric_path=metric_path,
        #         output_reverse=self._config["output_reverse"],
        #         de_lr=self._config["de_lr"],
        #         infod_lr=self._config["infod_lr"],
        #         crd_lr=self._config["crd_lr"],
        #         summary_freq=self._config["summary_freq"])
        #     gan.build()
        #     gan.load()

        #     results = {}

        #     factorVAEMetric_f = FactorVAEMetric(metric_data, sess=sess)
        #     factorVAEMetric_f.set_model(gan)
        #     results["FactorVAE"] = factorVAEMetric_f.evaluate(-1, -1, -1)

        #     betaVAEMetric_f = BetaVAEMetric(metric_data, sess=sess)
        #     betaVAEMetric_f.set_model(gan)
        #     results["betaVAE"] = betaVAEMetric_f.evaluate(-1, -1, -1)
            
            sapMetric_f = SAPMetric(metric_data, sess=sess)
            sapMetric_f.set_model(gan)
            results["SAP"] = sapMetric_f.evaluate(-1, -1, -1)

            # fStatMetric_f = FStatMetric(metric_data, sess=sess)
            # fStatMetric_f.set_model(gan)
            # results["FStat"] = fStatMetric_f.evaluate(-1, -1, -1)

            migMetric_f = MIGMetric(metric_data, sess=sess)
            migMetric_f.set_model(gan)
            results["MIG"] = migMetric_f.evaluate(-1, -1, -1)

            # for regressor in ["Lasso", "LassoCV", "RandomForest", "RandomForestIBGAN", "RandomForestCV"]:
            #     dciVAEMetric_f = DCIMetric(metric_data, sess=sess, regressor=regressor)
            #     dciVAEMetric_f.set_model(gan)
            #     results["DCI_{}".format(regressor)] = dciVAEMetric_f.evaluate(-1, -1, -1)

            # with open(os.path.join(self._work_dir, "final_metrics.pkl"), "wb") as f:
            #     pickle.dump(results, f)


